In [1]:
import pandas as pd
import glob

In [2]:
# import and read files
files  = glob.glob('detail\*.csv')
all_data = pd.DataFrame()
lis = [];
for f in files:
    df = pd.read_csv(f, encoding = "ISO-8859-1")
    lis.append(df)
    

C:\Users\Nitish\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# clean data
keep_cols = ['BIOGUIDE_ID', 'OFFICE','CATEGORY', 'PAYEE', 'START DATE', 'END DATE','PURPOSE','AMOUNT', 'YEAR']
for j in range(len(lis)):
    df2 = lis[j]
    if j==31:
        df2.drop('YEAR', axis=1, inplace=True)
        df2.columns = ['BIOGUIDE_ID','OFFICE','QUARTER','PROGRAM','CATEGORY','SORT SEQUENCE','DATE','TRANSCODE','PAYEE','START DATE','END DATE','PURPOSE','AMOUNT','YEAR']
    lis[j] = df2[keep_cols]
    

In [4]:
#merge data
all_data = pd.concat(lis)
all_data.index=(range(all_data.shape[0]))


In [5]:
#change type of amount to float
n= all_data['AMOUNT'].shape[0]
num = all_data['AMOUNT'].values
for i in range(n):
    if (',') in str(num[i]):
        num[i]=num[i].replace(',','')
        num[i]=float(num[i])
all_data['AMOUNT']=num

In [6]:
all_data[['AMOUNT']]=all_data[['AMOUNT']].apply(pd.to_numeric)


In [7]:
#total of transactions
total = all_data['AMOUNT'].sum()
print('{0:0.10f}'.format(total))


13660703793.3099994659


In [8]:
#change dates from string to date object
s = lambda x: str(x)
change_date = lambda x: x.replace(x, x[:len(x)-3]+'/20'+x[-2:]) if len(x)<9 and len(x)>=5 else x.lower()
all_data['END DATE'] = all_data['END DATE'].apply(s)
all_data['END DATE'] =  all_data['END DATE'].apply(change_date)
all_data['START DATE'] = all_data['START DATE'].apply(s)
all_data['START DATE'] =  all_data['START DATE'].apply(change_date)

In [9]:
all_data['END DATE'] = pd.to_datetime(all_data['END DATE'], format='%m/%d/%Y',errors='coerce')
all_data['START DATE'] = pd.to_datetime(all_data['START DATE'], format='%m/%d/%Y',errors='coerce')


In [10]:
#calculate covergae period
all_data['COVERAGE PERIOD'] = all_data['END DATE'] - all_data['START DATE']

In [11]:
all_data_positive = all_data.loc[all_data['AMOUNT']>=0]
coverage_std = all_data_positive['COVERAGE PERIOD'].std().total_seconds()/(3600*24)
print('{0:0.10f}'.format(coverage_std))


64.1342961250


In [12]:
#average expenditure for 7 years
df_7yrs_positive = all_data_positive[all_data_positive['START DATE'].notnull()]
df_7yrs_positive = df_7yrs_positive[df_7yrs_positive['START DATE'].isin(pd.date_range('2010-01-01','2016-12-31'))]
avg_expenditure_7yrs = df_7yrs_positive['AMOUNT'].sum()/7
print('{0:0.10f}'.format(avg_expenditure_7yrs))


1222767787.2285723686


In [13]:
#fraction of most expensive purpose of highest spending average to total expenditures starting in2016
df_2016 = all_data[all_data['START DATE'].isin(pd.date_range('2016-01-01','2016-12-31'))]
total_exp_2016 = df_2016['AMOUNT'].sum()
df_2016.groupby(['OFFICE']).sum().sort_values(by = ['AMOUNT'], ascending=False)
df_most_exp_office = df_2016[(df_2016['OFFICE'] == 'GOVERNMENT CONTRIBUTIONS')]
most_exp_purpose = df_most_exp_office.groupby(['PURPOSE']).sum().sort_values(by = ['AMOUNT'], ascending=False).values[0]
fraction = most_exp_purpose/total_exp_2016
print('{0:0.10f}'.format(fraction[0]))

0.0686516587


In [14]:
# change year to a number
change_year = lambda x: x.replace(x, x[-4:]) if len(x)>4 else x.lower()
all_data['YEAR'] = all_data['YEAR'].apply(s)
all_data['YEAR'] = all_data['YEAR'].apply(change_year)

In [15]:
all_data[['YEAR']]=all_data[['YEAR']].apply(pd.to_numeric, errors='coerce')


In [16]:
#highest staff salary among all represetatives in 2016
df_year = all_data[all_data['YEAR'].notnull()]
df_2016 = df_year[(df_year['YEAR']== 2016)]
df4 = df_2016[(df_2016['CATEGORY'] == 'PERSONNEL COMPENSATION')]
df5=df4.groupby(['BIOGUIDE_ID'])['AMOUNT'].sum()
df_payee=df4.groupby(['BIOGUIDE_ID'])['PAYEE'].unique().values
for i in range(len(df_payee)):
    df6 = df5/len(df_payee[i])
df6 = df6.sort_values(ascending=False)
print('{0:0.10f}'.format(df6[0]))

42844.6237777778


In [17]:
df_7yrs = df_year[(df_year['YEAR']>= 2010) & (df_year['YEAR'] < 2016)]
df_7yrs.head()

,BIOGUIDE_ID,OFFICE,CATEGORY,PAYEE,START DATE,END DATE,PURPOSE,AMOUNT,YEAR,COVERAGE PERIOD
123599,NaN,HOUSE INFORMATION RESOURCES,EQUIPMENT,10_07 P2 OFP1000024 CA...,2009-10-01,2009-10-01,EQUIPMENT MAINTENANCE,28560.00,2010.0,0 days
123615,NaN,CHIEF ADMINISTRATIVE OFFICER,EQUIPMENT,10_07 P2 OFP1000001 DO,2009-07-10,2009-07-10,EQUIPMENT MAINTENANCE,3169.77,2010.0,0 days
123616,NaN,CHIEF ADMINISTRATIVE OFFICER,EQUIPMENT,10_07 P2 OFP1000001 DO,2009-07-10,2009-07-10,EQUIPMENT MAINTENANCE,206873.18,2010.0,0 days
123617,NaN,CHIEF ADMINISTRATIVE OFFICER,EQUIPMENT,10_07 P2 OFP1000001 DO,2009-07-10,2009-07-10,EQUIPMENT MAINTENANCE,2842.77,2010.0,0 days
123618,NaN,CHIEF ADMINISTRATIVE OFFICER,EQUIPMENT,10_07 P2 OFP1000001 DO,2009-07-10,2009-07-10,EQUIPMENT MAINTENANCE,5685.53,2010.0,0 days
